In [1]:
import pandas as pd

In [2]:
import spacy
nlp = spacy.load('pl_spacy_model')

/home/kingus/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'pl_model' (0.1.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.4). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
comments_df = pd.read_csv("komentarze.csv")

In [4]:
comments_df.tail()

,article,comment,is_positive,thumbs_up,thumbs_down
4068,Nikodem Rozbicki zachwyca się Julią Wieniawą. ...,Jaki talent ma ta dziewczyna bo chyba jestem n...,0,15,0
4069,Nikodem Rozbicki zachwyca się Julią Wieniawą. ...,Chyba w rozkładaniu nóg i pracy ustami,0,9,3
4070,Nikodem Rozbicki zachwyca się Julią Wieniawą. ...,Jest wielkim talentem.... dobrze CIĄGNIE,0,8,2
4071,Nikodem Rozbicki zachwyca się Julią Wieniawą. ...,rany w czyn ten talent czyżby w jednym...........,0,4,2
4072,Nikodem Rozbicki zachwyca się Julią Wieniawą. ...,Gość co ma wyobraźnie o tablecie Pojechał moż...,0,7,0


In [5]:
import emoji
def remove_emoji(string):
    string = emoji.get_emoji_regexp().sub("", string)
    return string

In [6]:
import string
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus.reader import pl196x
import nltk
from nltk.corpus import stopwords
import spacy 

def clean_text(text):
    print(text)
    # lower text
    text = text.lower()
    
    # remove emoji
    text = remove_emoji(text)

    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]

    # remove stop words
    stops = get_stop_words('polish')
    text = [word for word in text if word not in stops]

    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    text = " ".join(text)
    
    for token in nlp(text):
        if(token.text!=token.lemma_):
            text = text.replace(token.text, token.lemma_)
    print(text)
    return(text)

comments_df["comments_clean"] = comments_df["comment"].apply(lambda x: clean_text(x))

Jakie to wszystko smutne. Plastikowy, sztuczny świat...
jaki smutny plastikowy sztuczny świat
haj Girlsss howa areyou! iam sooo goood. fajansiarstwo
haj girlsss howa areyou iam sooo goood fajansiarstwo
BOMBA dziewczyna buhaha wybuchłam
bomba dziewczyna buhaha wybuchłam
Żałosne są
żałosny
Kim ona jest ze ja ciagle promuja
kto z ciagle promuja
Jakas kpina
jakas kpina
Kto by pomyślał, że Oliwia będzie największą gwiazdą Tańca z Gwiazdami :D 
Pora zakończyć już ten format. 3-4 edycje ok, ale 25?????
by pomyśleć oliwia będzie wielki gwiazda taniec z gwiazda d 
porać zakończyć format edycja około ale
Piękna jest!
piękny
Ciekawe, który decydent "zaopiekuje się" nią?
ciekawy który decydent zaopiekować
A to Bomba się ustawia🤭🤭🤭🤭🤭zenujaca. 
a bomba ustawiazenujaca
Kim są Ci ludzie? Myślałam, że to ma być Taniec z Gwiazdami...
kto lud myślałam taniec z gwiazda
Dziewczę jest dorosłe i samo decyduje na co ma ochotę. A wy idealni w swoich idealnych światach i z idealnie wychowanymi dzieciątkami nie 

ciekawy polacy zdobyć chociaż medal ten igrzysko i w kategoria zdobywać medal fart
Skoro juz na koszt podatnikow robisz sobie wycieczke do Japonii to zdobadz jakis medal eh? Wielki szum o nic, nikogo to nie obchodzi. 
skoro juz koszt podatnikow robić sobie wycieczke japonii zdobadz jakis medal eh wielki szum o nikt obchodzić
Przesada 
przesada
Dramat !
dramat
I bardzo dobrze że nie mogą
i móc
A dlaczego nie można zawrzeć związku małżeńskiego z dwiema kobietami lub z dwoma mężczyznami, to jest dopiero dyskryminacja.
a można zawrzeć związek małżeński z dwa kobieta z dwa mężczyzna dopiero dyskryminacja
Kogo obchodzi jej życie osobiste. Ważniejsze jest jak reprezentowała Polskę na igrzyskach, słabo... Więc może w czasie takich zawodów wypadałoby się skupić na rywalizacji, albo sobie dać spokój z olimpiadą. 
kto obchodzić żyto osobisty ważny reprezentować polskę igrzysko słabo w czas taki zawód wypadałoby skupić rywalizacja sobie dać spokój z olimpiada
Ja nie wiem jak te celebrytki to robią

a zdawać meghan i harrego wstrętny brytyjski dwór tortura zmusić zgoda dopisanie by w życie dobrowolnie zgodzić przecież darmo stamtąd uciec za wielki woda i odciąć ten paskudny monarchia
Doda przyznała że kłamała w żywe oczy w swoim wywiadzie u Janachowskiej nt "sekretnego ślubu". Już wtedy rozstała się z mężem. A to nie ona czasem naskakiwała na wszystkich tłumacząc że tylko ona jest autentyczna i nie owija w bawełnę? 😂 Pudel nie napisze o tym ale ja wam powiem 😀 
dodać przyznać kłamać w żywy oko w swój wywiad u janachowskiej nt sekretny ślub rozstać z mąż a czas naskakiwać wszystek tłumaczyć tylko autentyczny i owijać w bawełna pudel napisać o to ale powiedzieć
Coco jambo i do przodu,  byle do przodu, po trupach 
coca jambo i przód byle przód trupa
Reprezentują royalizm, ale w przystępnej, zrozumiałej dla mas, trochę śmiesznej formie. Swoim życiem nie wpędzają w kompleksy. Są nieporadni i łatwo poczuć się przy nich lepiej. Relaksik dla niewymagających jakości.
reprezentować royalizm

anka w grób przewracać
Tatuś czemu nie zatańczy ?
tatuś czemu zatańczyć
Po co ona tam ?

Nowy Brezniew z tymi brwiami
nowy brezniew z ten brew
W życiu bym tego nie obejrzała... jakieś dzieci wątpliwych gwiazd to już dno. A brwi ma ohydne, zresztą lansuje się dziecko tak nachalnie, że wstydziła by się 
w życie bym to obejrzeć jakiś dziecko wątpliwy gwiazda dna a brew ohydny zresztą lansować dziecko nachalnie wstydzić by
Szkoda, że nie ma Sylwii Madeńskiej wśród tancerzy. Mogła dostać jeszcze jedną szansę bo odpadła tak wcześnie tylko przez drewnianego Mikołaja :/
szkoda sylwii madeńskiej wśród tancerz móc dostać jeszcze jeden szansa odpaść wcześnie tylko przez drewniany mikołaj
To komin będzie czyszczony .
komin będzie czyścić
faktycznie "gwiazda"...
faktycznie gwiazda
Nie będę wchodziła na info o Bieniuk bo na siłę robicie z niej gwiazdę. A jest zwyklą dziewczynką, bez odkrytych talentów (może je ma) i nic jeszcze nie osiągnęła. 
będę wchodzić info o bieniuk siła robić z gwiazda a zwyk

chodzić w ciąża będzie z miesiąc
Kobieta tuż przed rozwiązaniem, w 40 stopniowym upale. Nie wiem czego się fotografina spodziewał, miss mokrego podkoszulka? Ma fajną, przewiewną kieckę i wygodne buty. Dokładnie tak, jak powinna.
kobieta tuż rozwiązanie w stopniowy upał wiedzieć co fotografina spodziewać miss mokry podkoszulek fajny przewiewny kiecka i wygodny buta dokładnie powinna
niezależnie od tego czy w ciąży czy nie w ciąży- ona po prostu jest jakaś odpychająca, niechlujna  i irytująca
niezależnie to w ciąża w ciąża prosty jakiś odpychający chlujny i irytujący
Jak się je takie zapiekany długie na kilometr to nic dziwnego, że się tyje. Nie ma co wszystkiego zwalać na ciążę. Nie można sobie folgować, ot co!.- oczywiście jeśli chce się zachować figurę. Po prostu trzeba się pilnować i nie jeść za 10. 
taki zapiekany długi kilometr dziwny tyć wszystko zwalać ciąża można sobie folgować ot oczywiście chcieć zachować figura prosty trzeba pilnować i jeść za
Zdjecie nr.6 mówi wszystko ,Bard

...

dziwnie wyglada stary samotny kobieta,az zal
napewno wie, co sobie wstrzykuje w twarz, bo przeciez gdyby nie wiedziala, to by sobie nie wstrzykiwala, czyli to jest napewno zdrowe i bezpieczne
napewno wiedzieć sobie wstrzykiwać w twarz przeciez wiedziećdziala by sobie wstrzykiwala czyli napewno zdrowy i bezpieczny
A co ma siedziec w domu? xD 
a siedziec w dom xd
Może niech trochę uwagi poświęci synowi
niech trochę uwaga poświęcić syn
To ona ma znajomych??????????????????????
znajomy
Uwielbiam Edytkę. Kiniaczka i Jej córki. Dzisiaj piątek, więc tankuję do odcięcia. A co tam. 😁☝️😂
uwielbiać edytkę kiniaczka i córka piątka tankować odcięcia a ️
A ten z długimi włosami to nie Krupa? 🤔
a z długi włos krupa
a pani Edzia tak do ust bierze coś czego sama nie wyprodukowała ze znanych sobie składników? Tak po prostu zaufała jakiemuś barmanowi, który nalał jej coś z butelki, na której była jakaś etykieta, którą każdy może sobie wydrukować? Taka bezmyślność u osoby, która nie ufa firmom farmaceutyc

ale czemu umarl wyglądać ostatnio chory
A co to nas obchodzi? czy to ktoś wazny dla społeczenstwa ,kultury,nauki?  Nie macie o kim pisac?
a obchodzić ktoś wazny społeczenstwa kultura,nauka mieć o kto pisac
Wyrazy współczucia dla Lary...byłaś kochana i kochającą córką,nie martw się tata będzie Cię wspierał,czego Ci życzę.
wyraz współczucie lar...byłaś kochany i kochać córka,nie martwić tata będzie wspierać,co życzyć
Najcudowniejszy, bo zajął się córką, aby jej matka mogła skakać po innych rozporkach
cudowny zająć córka aby matka móc skakać inny rozporek
Ta pani niech przestanie zakładać ta perukę na głowę bo to nie są jej włosy i niech się nie upodabnia do pani Wilkasy bo to nie ten sort
ten pani niech przestennie zakładać ten peruka głowa włos i niech upodabniać pani wilkasy sorta
Jakoś w przypadku Piaska czy Szpaka interesowało ich to co poza sceną i przez to panowie stracili pracę w wVOP
jakoś w przypadek piaska szpak interesować poza scena i przez pan stracić praca w wvop
no cóz inn

skoro z synek i z taki rozrzwnienie wspominać sielski dzieciństwo warto by być wspomnieć ojczym oprócz trzy wpatrzyć w kobieta mieć też rodzony syn mikołaj stary brat lar z strona ojciec który matka uosobienie klasa i godność osobisty marta gessler pewno przyszywać potomek przz całkowity przypadek zapomnieć o istnienie nieodrodny syn swój matka
Raptem 63 lata, uznanie za opiekę nad dziećmi.
raptem latać uznanie za opieka nad dziecko
Skoro opiekował się córką i przybranym  synem, musiał być dobrym człowiekiem.A gdzie była MG?
skoro opiekować córka i przybranym syn musieć dobry człowiekiem.a być miligram
Jak sama nazwa wskazuje biustonosz nosi się aby nosił biust... U tego pana ma szokować, nic nie symbolizuje... Chcę aby coś się działo i tyle... 
nazwa wskazywać biustonosz nosić aby nosićł biust u ten pan szokować symbolizować chcieć aby coś dziać i tyle
No u kobiety to chyba jasne do czego sluzy. A u faceta? Tez chetnie sie dowiem...
no u kobieta chyba jasny co sluzy a u facet tez chet

łaska robić dać spokój święty znaleźć
eeee tam, znajdzie sobie młodszą. Facet nie ma z tym problemu.
eeee znaleźć sobie młody facet z to problem
Dlaczego oni nie pójdą na terapię dla par 🙄🤗
pójść terapia para
Ona go powinna pozwać za ten belkot
powinna pozwać za belkot
Niech im ktoś pomoże a nie fotki na pudlu co chwila 
niech ktoś pomóc a fotka pudel chwila
Pasujecie do siebie jak z obrazka!!!!Tak trudno dojsc    do porozumienia!!!!?Jestescie piekna para.
pasować siebie z obrazka!!!!tak trudno dojsc porozumienia!!!!?jestescie piekna par
biedna Ilona i Hannah Montana... :(
biedny ilona i hannah montana
Piłeś? Nie pisz...
piłeś pisać
Smutne. Dwoje emocjonalnie niestabilnych alkoholików stęsknionych za „sławą” sprzedaje swoje życie szmatławcom za zapewne śrubki. Przykro patrzeć.
smutny emocjonalnie stabilny alkoholik stęsknionych za „sława” sprzedawać swój żyto szmatławiec za zapewne śrubka przykro patrzeć
jedno trzeba przyznać - jak na swój wiek i tryb życia facet wygląda bardzo dobrze.

pięknie torebka cudowny
Sukienka ładna, ja bym rozpięła pod szyją , byłoby wygodniej, krawacik dziwny, ale myślę ,że miało być śmiesznie, włosy normalne.
sukienko ładny bym rozpiąć szyja byłoby wygodno krawacik dziwny ale myślić mieć śmiesznie włos normalny
Za to buty mega fajne! Sztos !
za buta mega fajny sztos
Piekna naturalna kobieta, wyglada bardzo dobrze , a o gustach sie nie dyskutuje
piekna naturalny kobieta wyglada a o gust sie dyskutować
Koszmar te doczepy sukienka dramat A taka ładna na wspólnej 
koszmar ten doczepa sukienko dramat a taki ładny wspólny
Ona nie chce byc przasna a jest niestety
chcieć byc przasna a niestety
Kiecka kiepska, fakt. Włosy normalne.
kiecka kiepski fakt włos normalny
Fajnie wygląda, zupełnie inaczej niż te napompowane lale, brawo za naturalność i własny styl
fajnie wyglądać zupełnie inaczej ten napompować lala brawo za naturalność i własny stylo
Abstrahując od taktu: stwierdzenie, że sukienka jest brzydka, tonie jest hejt! Byłoby nim obraźliwe wypowi

In [7]:
comments_df["comments_clean"].head()

0                jaki smutny plastikowy sztuczny świat
1    haj girlsss howa areyou iam sooo goood fajansi...
2                    bomba dziewczyna buhaha wybuchłam
3                                              żałosny
4                                 kto z ciagle promuja
Name: comments_clean, dtype: object

In [8]:
comments_df["is_positive"].value_counts()

0                                                                                                     3583
1                                                                                                      484
  takie same rzeczy produkowane przez male raczki tylko z wlasciwa metka i tansze niz w dyskontach       1
p                                                                                                        1
) I my mamy to kupić?                                                                                    1
)                                                                                                        1
)                                                                                                        1
!!!!!!                                                                                                   1
Name: is_positive, dtype: int64

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
dv = vectorizer.fit_transform(comments_df["comments_clean"])

In [10]:
# vectorizer.get_feature_names()

In [11]:
# print(vectorizer.vocabulary_)

In [12]:
dshuf = comments_df.sample(frac = 1)

In [13]:
dtrain = dshuf[:1800]
dtest = dshuf[1800:]
dtrain_att = vectorizer.fit_transform(dtrain['comments_clean'])
dtest_att = vectorizer.transform(dtest['comments_clean'])
dtrain_label = dtrain['is_positive']
dtest_label = dtest['is_positive']

In [14]:
dtest.count()

article           2273
comment           2273
is_positive       2273
thumbs_up         2273
thumbs_down       2273
comments_clean    2273
dtype: int64

In [15]:
dtrain.count()

article           1800
comment           1800
is_positive       1800
thumbs_up         1800
thumbs_down       1800
comments_clean    1800
dtype: int64

In [16]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 50, random_state = 0)
RFC.fit(dtrain_att, dtrain_label)

RandomForestClassifier(n_estimators=50, random_state=0)

In [17]:
print(comments_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4073 entries, 0 to 4072
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   article         4073 non-null   object
 1   comment         4073 non-null   object
 2   is_positive     4073 non-null   object
 3   thumbs_up       4073 non-null   int64 
 4   thumbs_down     4073 non-null   int64 
 5   comments_clean  4073 non-null   object
dtypes: int64(2), object(4)
memory usage: 191.0+ KB
None


In [18]:
RFC.score(dtrain_att, dtrain_label)

0.9994444444444445

In [19]:
from sklearn.metrics import confusion_matrix
y_pred = RFC.predict(dtrain_att)
confusion_matrix(y_pred, dtrain_label)

array([[   1,    0,    0,    0,    0],
       [   0,    1,    0,    0,    0],
       [   0,    0, 1577,    1,    0],
       [   0,    0,    0,  219,    0],
       [   0,    0,    0,    0,    1]])

In [20]:
from sklearn.pipeline import Pipeline,make_pipeline
pl = Pipeline([
 ('bag of words: ', CountVectorizer()),
 ('Random Forest Classifier:', RandomForestClassifier())])
make_pipeline(CountVectorizer(), RandomForestClassifier())

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [21]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(RFC, dtrain_att, dtrain_label, cv=5)
scores.mean()

/home/kingus/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


0.903888888888889

In [22]:
dshuf = comments_df.sample(frac=1)
d_content = dshuf['comments_clean']
d_label = dshuf['is_positive']

In [23]:
d_label[:149]

2901    0
463     0
359     0
2942    0
2314    0
       ..
117     0
887     0
247     0
2576    0
3872    0
Name: is_positive, Length: 149, dtype: object

In [24]:
d_content[:149]

2901    łatwo kochać ktoś potrzebować to polegać miłoś...
463            mama znany troche nieznany ale pewno lubić
359            ciekawy wygladala kiedys,za czasow slowika
2942    każdy rozstanie boleć ale trzeba zachować klas...
2314    juz widzieć ten nagłówek będą rozstawać i nagl...
                              ...                        
117     musić mieść rojalista jakiś hak wywijać a elżb...
887                                 matka tyłek koszmarny
247                             myślałam będzie coś dobry
2576                                       i sobie zrobić
3872    niezastąpiony nazwisko ciągły mieć mimo tyle r...
Name: comments_clean, Length: 149, dtype: object

In [25]:
pl.fit(d_content[:4068],d_label[:4068])

Pipeline(steps=[('bag of words: ', CountVectorizer()),
                ('Random Forest Classifier:', RandomForestClassifier())])

In [26]:
pl.score(d_content[:4068],d_label[:4068])

0.9992625368731564

In [27]:
pl.predict(["jest słaba, przestańcie o niej pisać"])

array(['0'], dtype=object)

In [28]:
pl.predict(["lubię ją! super babka."])

array(['1'], dtype=object)

In [29]:
pl.predict(["super jest! uwielbiam ją!"])

array(['1'], dtype=object)

In [30]:
pl.predict(["jest śliczna! super wygląda"])

array(['1'], dtype=object)

In [31]:
pl.predict(["żałosna, nie lubię jej."])

array(['0'], dtype=object)

In [32]:
pl.predict(["wygląda ślicznie. ma super figurę. widać, że jest szczęśliwa!"])

array(['1'], dtype=object)

In [33]:
pl.predict(["wygląda staro, nie mogę na nią patrzeć"])

array(['0'], dtype=object)

In [34]:
pl.predict(["Gratulacje! Będą mieli śliczne dzieci!"])

array(['0'], dtype=object)

In [35]:
pl.predict(["Nie mogę na nich patrzeć, okropnie wygląda w tym kostiumie!"])

array(['0'], dtype=object)

In [36]:
pl.predict(["Wreszcie normalna, fajna babka!"])

array(['1'], dtype=object)

In [37]:
pl.predict(["Ile można o nich pisać?"])

array(['0'], dtype=object)

In [38]:
pl.predict(["No i brawo! Wreszcie zwycięstwo! Pokazali im!"])

array(['1'], dtype=object)

In [39]:
pl.predict(["Fajna para"])

array(['0'], dtype=object)

In [40]:
pl.predict(["Co za źle dobrana para. Powinni się rozstać."])

array(['0'], dtype=object)

In [41]:
pl.predict(["Cudowni, gratulacje. Oby byli szczęśliwi"])

array(['0'], dtype=object)

In [42]:
pl.predict(["Fajny dzieciak. Gratulacje"])

array(['1'], dtype=object)

In [43]:
pl.predict(["Fajny dzieciak. Gratulować"])

array(['1'], dtype=object)